In [83]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt5
df = pd.read_csv('SCRLOG_02.csv', sep =';')
df.columns = df.columns.str.strip()

#count = df['wind_correct.heading'].isnull().sum()
#df=df.drop([0, count-1]).reset_index(drop=True)
#df = df[:-count].reset_index(drop=True)

df = df.drop_duplicates().reset_index(drop=True)
df=df[1:].reset_index(drop=True)

empty_numbers=df[df['wind_correct.heading'].isnull()].index.tolist()
df=df.dropna(axis=0, how='any')#не запустила в первом случае
df = df.apply(pd.to_numeric, errors='coerce')#учла наличие пустых значений
empty_numbers.append(len(df))

df = df[['wrap_360(math.deg(current_orient:get_euler_yaw()))','calc_target.heading']]
df.info

<bound method DataFrame.info of       wrap_360(math.deg(current_orient:get_euler_yaw()))  calc_target.heading
0                                              253.8388              247.9940
1                                              253.9931              247.9940
2                                              254.1311              247.9940
3                                              254.2680              247.8018
4                                              254.3796              247.8018
...                                                 ...                   ...
3545                                           344.5722              338.8967
3546                                           344.6370              338.8967
3547                                           344.6926              339.9531
3548                                           344.7438              339.9850
3549                                           344.7961              340.0076

[3546 rows x 2 columns]>




-- Оптимальный угол ветра 
function OptimizeAngleWind(Plane_curse, Wind_curse)
	local X_plane, X_wind
	local Y_plane, Y_wind
	local Z_plane, Z_wind
	local Angle_wind
	X_plane, Y_plane, Z_plane = vector(Plane_curse)
	X_wind, Y_wind, Z_wind = vector(Wind_curse)
	local sinus_part = X_plane*X_wind + Y_plane*Y_wind
	--local cosinus_part = X_plane*Z_wind - Z_plane*X_wind + Y_plane*Z_wind - Z_plane*Y_wind + X_plane*Y_wind - X_wind*Y_plane -- универсальная, но длинная
	local cosinus_part = X_plane*Y_wind - X_wind*Y_plane
	Angle_wind = math.atan2(cosinus_part, sinus_part)
	return Angle_wind
end

In [39]:
################
import numpy as np
x_plane = y_plane = x_point = y_point = df
x_plane['wrap_360(math.deg(current_orient:get_euler_yaw()))'] = np.sin(df['wrap_360(math.deg(current_orient:get_euler_yaw()))'])
x_plane

,wrap_360(math.deg(current_orient:get_euler_yaw())),calc_target.heading
0,0.482470,247.9940
1,0.403241,247.9940
2,0.309184,247.9940
3,0.194512,247.8018
4,0.088814,247.8018
...,...,...
3545,-0.587678,338.8967
3546,-0.575926,338.8967
3547,-0.564027,339.9531
3548,-0.551393,339.9850


In [73]:
cosinus_part = pd.DataFrame(columns = ['cos'])
cosinus_part['cos'] = x_plane['wrap_360(math.deg(current_orient:get_euler_yaw()))']*y_point['calc_target.heading'] - x_point['calc_target.heading']*y_plane['wrap_360(math.deg(current_orient:get_euler_yaw()))']
cosinus_part

,cos
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3545,0.0
3546,0.0
3547,0.0
3548,0.0


In [82]:
x_plane['wrap_360(math.deg(current_orient:get_euler_yaw()))']*y_point['calc_target.heading']

0       0.994180
1       0.994176
2       0.994173
3       0.994174
4       0.994172
          ...   
3545    0.989241
3546    0.989239
3547    0.989204
3548    0.989202
3549    0.989199
Length: 3546, dtype: float64

In [80]:
x_point['calc_target.heading']*y_plane['wrap_360(math.deg(current_orient:get_euler_yaw()))']

0       0.994180
1       0.994176
2       0.994173
3       0.994174
4       0.994172
          ...   
3545    0.989241
3546    0.989239
3547    0.989204
3548    0.989202
3549    0.989199
Length: 3546, dtype: float64

In [79]:
import math as m
import numpy as np
x_plane = y_plane = x_point = y_point = df
sinus_part = pd.DataFrame(columns = ['sinus_part'])
cosinus_part = pd.DataFrame(columns = ['cosinus_part'])
angle = pd.DataFrame(columns = ['angle'])
df['wrap_360(math.deg(current_orient:get_euler_yaw()))'] = np.radians(df['wrap_360(math.deg(current_orient:get_euler_yaw()))'])

x_plane['wrap_360(math.deg(current_orient:get_euler_yaw()))'] = np.sin(df['wrap_360(math.deg(current_orient:get_euler_yaw()))'])
y_plane['wrap_360(math.deg(current_orient:get_euler_yaw()))'] = np.cos(df['wrap_360(math.deg(current_orient:get_euler_yaw()))'])

df['calc_target.heading'] = np.radians(df['calc_target.heading'])
x_point['calc_target.heading'] = np.sin(df['calc_target.heading'])
y_point['calc_target.heading'] = np.cos(df['calc_target.heading'])

sinus_part['sinus_part'] = x_plane['wrap_360(math.deg(current_orient:get_euler_yaw()))']*y_point['calc_target.heading'] + y_plane['wrap_360(math.deg(current_orient:get_euler_yaw()))']*y_point['calc_target.heading']
cosinus_part['cosinus_part'] = x_plane['wrap_360(math.deg(current_orient:get_euler_yaw()))']*y_point['calc_target.heading'] - x_point['calc_target.heading']*y_plane['wrap_360(math.deg(current_orient:get_euler_yaw()))']
angle['angle'] = np.arctan2(sinus_part['sinus_part'], cosinus_part ['cosinus_part'])
print(cosinus_part)
print(sinus_part)

      cosinus_part
0              0.0
1              0.0
2              0.0
3              0.0
4              0.0
...            ...
3545           0.0
3546           0.0
3547           0.0
3548           0.0
3549           0.0

[3546 rows x 1 columns]
      sinus_part
0       1.988360
1       1.988352
2       1.988346
3       1.988348
4       1.988343
...          ...
3545    1.978481
3546    1.978477
3547    1.978408
3548    1.978403
3549    1.978398

[3546 rows x 1 columns]
